In [423]:
"""import the required libraries"""
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

In [424]:
"""load data into the system"""
# data = np.loadtxt(open("/Users/kavinarasu/Desktop/housevotes.csv", "rb"), delimiter=",")
data = pd.read_csv("/Users/kavinarasu/Desktop/housevotes.csv",header = None)

In [425]:
"""shuffle and split the data"""
data = shuffle(data)
data = data.values
Train,Test = train_test_split(data,test_size=0.2,random_state=17)


In [448]:
def entropy_2att(a,b):
    if a == 0 or b==0:
        return 0
    else:
        pdemo = (a/(a+b))
        prepub = (b/(a+b))
        entropy = -(pdemo*np.log2(pdemo))-(prepub*np.log2(prepub))
        return entropy
def call_info_gain(traindata):
    """calculating global entropy"""
    demo = 0
    repub = 0
    for k in range(traindata.shape[0]):
        if traindata[k][0] == 'democrat':
            demo=demo+1
        else:
            repub=repub+1
    glb_entropy = entropy_2att(demo,repub)
    gain = []
    entropy = []
    for i in range (1,traindata.shape[1]):
        ycount = 0
        ncount = 0
        nvcount = 0
        ydemo = 0
        yrepub = 0 
        ndemo = 0
        nrepub = 0 
        nvdemo = 0
        nvrepub = 0
        for j in range (traindata.shape[0]):
            if(traindata[j][i]=='y'):
                ycount = ycount+1
                if(traindata[j][0]=='democrat'):
                    ydemo = ydemo+1
                else:
                    yrepub = yrepub+1
            elif(traindata[j][i]=='n'):
                ncount = ncount+1
                if(traindata[j][0]=='democrat'):
                    ndemo = ndemo+1
                else:
                    nrepub = nrepub+1
            elif(traindata[j][i]=='?'):
                nvcount = nvcount+1
                if(traindata[j][0]=='democrat'):
                    nvdemo = nvdemo+1
                else:
                    nvrepub = nvrepub+1
#         print(ycount,ncount,nvcount,ydemo,yrepub,ndemo,nrepub,nvdemo,nvrepub)
        entropy = np.append(entropy,((ycount/traindata.shape[0]) * entropy_2att(ydemo,yrepub)) + ((ncount/traindata.shape[0]) * entropy_2att(ndemo,nrepub)) + ((nvcount/traindata.shape[0]) * entropy_2att(nvdemo,nvrepub)))
        gain = np.append(gain, glb_entropy - entropy[i-1])
#     print("printing",entropy)
    return gain

"""tree defenition"""

class Tree:
    def __init__(self, parent=None):
        self.parent = parent
        self.children = []
        self.splitFeature = None
        self.splitFeatureValue = None
        self.label = None
        self.classcount = None
        
def checkmajority(traindata,root):
    democnt=0
    repubcnt=0
    for i in range(traindata.shape[0]):
        if(traindata[i][0]=='democrat'):
            democnt=democnt+1
        else:
            repubcnt=repubcnt+1
    if(repubcnt>democnt):
        root.label="republican"
    else:
        root.label="democrat"
        
        
        
def decisionTree(traindata):
    return build_decision_tree(traindata,Tree(),set(range(16)))

def build_decision_tree(traindata,root,remaining_feature):

    if(leafcheck(traindata)):
#         print("entered")
#         root.classcount = traindata.shape[0]
        root.label = traindata[0][0]
        return root
    if(leafcheck(traindata) == False):
        checkmajority(traindata,root)        
    bestsplitvalue = 0
#     print(traindata.shape[0])
    temp = np.argsort(call_info_gain(traindata))
    for i in reversed(temp):
#         print(i)
        if i in remaining_feature:
            bestsplitvalue=i
            break
    
    root.splitFeature =  bestsplitvalue
    
    for datasubset in splitdata(traindata,i):
        aChild = Tree(parent=root)
        aChild.splitFeatureValue = datasubset[0][bestsplitvalue+1]
        root.children.append(aChild)
        build_decision_tree(datasubset, aChild, remaining_feature - set([bestsplitvalue]))
    return root
#         print(datasubset)
#     for datasubset in splitdata(traindata,root.splitFeature):

        
def splitdata(traindata,bestFeature):
#     print(bestFeature)
    subset = []
    a=0
    for i in range(traindata.shape[0]):
        if (traindata[i][bestFeature+1]=='y'):
            a=a+1
            subset = np.append(subset,traindata[i][:])
        subset = np.array(subset)
        subset = subset.reshape(a,17)
    if subset.shape[0] !=0:
        yield subset
    
    subset = []
    a=0
    for i in range(traindata.shape[0]):
        if (traindata[i][bestFeature+1]=='n'):
            a=a+1
            subset = np.append(subset,traindata[i][:])
        subset = np.array(subset)
        subset = subset.reshape(a,17)
    if subset.shape[0] !=0:
        yield subset

    subset = []
    a=0
    for i in range(traindata.shape[0]):
        if (traindata[i][bestFeature+1]=='?'):
            a=a+1
            subset = np.append(subset,traindata[i][:])
        subset = np.array(subset)
        subset = subset.reshape(a,17)
    if subset.shape[0] !=0:
        yield subset

    
    
def leafcheck(traindata):
    group = traindata[:,0]
    return all(x==group[0] for x in group)


            
def predict(row,root,temp):
    if root.children == []:
        if (root.label == None):
            return temp.label
        else:
            return root.label
    
    else:
#         print(root.label)
        i = root.splitFeature
#         print( i)
        value = row[i+1];
#         print(value)    
        no_of_child = len(root.children)
        for j in range(no_of_child):
            if (root.children[j]).splitFeatureValue == value:
#                 print(root.children[j].splitFeatureValue)
                return predict(row,root.children[j],root)
            else:
                return root.label

def predicttest(row,root,temp):
    if not root.children:
        if (root.label == None):
            return temp.label
        else:
            return root.label
    
    else:
#         print(root.label)
        i = root.splitFeature
#         print( i)
        value = row[i+1];
#         print(value)    
        no_of_child = len(root.children)
        for j in range(no_of_child):
            if (root.children[j]).splitFeatureValue == value:
#                 print(root.children[j].splitFeatureValue)
                return predicttest(row,root.children[j],root)
            else:
                return root.label


# call_info_gain(Train)
tree = decisionTree(Train)
# printBinaryDecisionTree(tree)
# prediction = predict(Test[1][:],tree)
# print(prediction)
predicted=[]
train_predict=[]
for i in range(Test.shape[0]):
    predicted=np.append(predicted,predict(Test[i][:],tree,None))

for i in range(Train.shape[0]):
    train_predict=np.append(train_predict,predict(Train[i][:],tree,None))
    
def printdtree(root):
    print(root.splitFeature,root.splitFeatureValue,root.label)
    if root.children:
        
        if(len(root.children) == 1):
#             print(len(root.children))
            printdtree(root.children[0])
        if(len(root.children) == 2):
#             print(len(root.children))
            printdtree(root.children[0])
            printdtree(root.children[1])
        if(len(root.children) == 3):
#             print(len(root.children))
            printdtree(root.children[0])
            printdtree(root.children[1])
            printdtree(root.children[2])

def prunetest(root,tain_accuracy):
#     print(root.splitFeatureValue,root.splitFeature,root.label)
    newtrain_predict =[]
    if root.children:
        if (len(root.children) == 1):

        #             print(len(root.children))
            prunetest(root.children[0],train_accuracy)
            root.children= None
            for i in range(Train.shape[0]):
                newtrain_predict=np.append(newtrain_predict,predicttest(Train[i][:],root,None))
            newtrain_accuracy=accuracy_score((Train[:,0]),(newtrain_predict))
#             print(root.splitFeatureValue,root.splitFeature,root.label)
            print(newtrain_accuracy)
#             printdtree(root)
            if(newtrain_accuracy > tain_accuracy):
                print("we done it")

        if (len(root.children) == 2):
        #             print(len(root.children))
            prunetest(root.children[1],train_accuracy)
            root.children= None
            for i in range(Train.shape[0]):
                newtrain_predict=np.append(newtrain_predict,predicttest(Train[i][:],root,None))
            newtrain_accuracy=accuracy_score((Train[:,0]),(newtrain_predict))
#             print(root.splitFeatureValue,root.splitFeature,root.label)
            print(newtrain_accuracy)
#             printdtree(root)
            if(newtrain_accuracy > tain_accuracy):
                print("we done it")
        if (len(root.children) == 3):
        #             print(len(root.children))
            prunetest(root.children[2],train_accuracy)
            root.children= None
            for i in range(Train.shape[0]):
                newtrain_predict=np.append(newtrain_predict,predicttest(Train[i][:],root,None))
            newtrain_accuracy=accuracy_score((Train[:,0]),(newtrain_predict))
#             printdtree(root)
            print(newtrain_accuracy)
#             print(root.splitFeatureValue,root.splitFeature,root.label)
            if(newtrain_accuracy > tain_accuracy):
                print("we done it")
# printdtree(tree)
# print(predicted)
# print(Test[:,0])
train_accuracy = accuracy_score((Train[:,0]),(train_predict))
print("testing accuracy",accuracy_score((Test[:,0]),(predicted)))
print("training accuracy",accuracy_score((Train[:,0]),(train_predict)))

prunetest(tree,train_accuracy)



testing accuracy 0.9310344827586207
training accuracy 0.9683908045977011
0.6235632183908046
0.6235632183908046


In [349]:
(Test[:,0])

array(['republican', 'democrat', 'republican', 'democrat', 'republican',
       'republican', 'democrat', 'republican', 'republican', 'republican',
       'republican', 'republican', 'democrat', 'democrat', 'democrat',
       'democrat', 'democrat', 'democrat', 'democrat', 'republican',
       'democrat', 'democrat', 'democrat', 'democrat', 'democrat',
       'democrat', 'republican', 'republican', 'democrat', 'democrat',
       'democrat', 'democrat', 'democrat', 'republican', 'democrat',
       'republican', 'democrat', 'republican', 'democrat', 'democrat',
       'democrat', 'democrat', 'republican', 'democrat', 'democrat',
       'democrat', 'democrat', 'democrat', 'democrat', 'republican',
       'republican', 'republican', 'democrat', 'democrat', 'democrat',
       'democrat', 'republican', 'republican', 'democrat', 'democrat',
       'democrat', 'republican', 'republican', 'democrat', 'democrat',
       'republican', 'democrat', 'democrat', 'republican', 'democrat',
       'demo